In [1]:
import np
import re
from functools import reduce
import pandas as pd

# Task 1

In [37]:
while True:
    try:
        code_type = input("ELECTRONIC MASTERMIND 1977\nSELECT CODE TYPE: [3, 4, 5]: ").strip()
        if code_type.lower() == "stop":
            print("Terminated", end="\n\n\n")
            break
        N = int(code_type)
        if N not in [3, 4, 5]: raise ValueError()
    except ValueError:
        print("Not a valid code type.", end="\n\n\n")
    else:
        shield = np.random.choice(10, N)
        print(shield, "NOTE: THIS SHOULD BE HIDDEN. IT IS SHOWN FOR DEMO PURPOSES ONLY")
        i = 0
        output = np.array(["."]*N)
        while (output != "B").any():
            try:
                ui = np.array(re.split(r"\s", input("Enter the code separated by space: "))).astype(int)
                if len(ui) != N: raise ValueError("Insufficient Values in Input")
            except ValueError as e:
                print(e if len(str(e)) and not str(e).startswith("invalid literal for int()") else "Unexpected Format for Sequence\n")
            else:
                output = np.array(["."]*N)
                output[np.isin(ui, list(set(shield[ui != shield]) & set(ui[ui != shield])))] = "W"
                output[ui==shield] = "B"
                i += 1
                print("Guess", i, ":", np.sum(output=="B"), np.sum(output=="W"), output,  end="\n\n")

        print("SOLVED:", shield, end="\n\n\n")

ELECTRONIC MASTERMIND 1977
SELECT CODE TYPE: [3, 4, 5]: 3
[6 0 9] NOTE: THIS SHOULD BE HIDDEN. IT IS SHOWN FOR DEMO PURPOSES ONLY
Enter the code separated by space: 1 2 6
Guess 1 : 0 1 ['.' '.' 'W']

Enter the code separated by space: 6 3 0
Guess 2 : 1 1 ['B' '.' 'W']

Enter the code separated by space: 6 0 4
Guess 3 : 2 0 ['B' 'B' '.']

Enter the code separated by space: 6 0 5
Guess 4 : 2 0 ['B' 'B' '.']



KeyboardInterrupt: Interrupted by user

In [ ]:
N = int(input("ELECTRONIC MASTERMIND 1977\nSELECT CODE TYPE: [3, 4, 5]: ").strip())
shield = np.random.choice(10, N)
output = np.array(["."]*N)

dataset = {i:[] for i in range(10)}

while (output != "B").any():
    ui = np.array(re.split(r"\s", input("Enter the code separated by space: "))).astype(int)
    

In [2]:
def mark(shield, guess):
    output = np.full(len(shield), ".", dtype=str)
    output[np.isin(guess, list(set(shield[guess != shield]) & set(guess[guess != shield])))] = "W"
    output[guess==shield] = "B"
    return output


def score(shield, guess):
    return np[(guess==shield).sum(), np.isin(guess, list(set(shield[guess != shield]) & set(guess[guess != shield]))).sum()]

In [3]:
def crossover(code1, code2):
    return np.where(np.random.random(size=len(code1))<0.5, code1, code2)

def mutate(code):
    code[np.random.randint(len(code)-1)] = np.random.randint(9)
    return code

def permute(code):
    for i in range(len(code)):
        if np.random.random() < 0.03: # very low
            a, b = np.random.randint(len(code)-1, size=2)
            code[a], code[b] = code[b], code[a]
    
    return code

def inversion(code):
    code[0], code[-1] = code[-1], code[0]
    return code

In [15]:
crossover([1,2,3],[3,2,1])

array([3, 2, 3])

In [17]:
mutate([1,2,3])

[1, 1, 3]

In [23]:
permute([1,2,3])

[1, 2, 3]

In [30]:
np.random.choice(10, size=(150, 3)).sum(axis=0)

array([720, 679, 630])

In [4]:
def generatePopulation(size=(150,3)):
    pop = np.random.choice(10, size=size)
    df = pd.DataFrame(pop).drop_duplicates()
    while len(df) < size[0]:
        pop = np.random.choice(10, size=(size[0]-len(df), size[-1]))
        df = pd.concat([df, pd.DataFrame(pop)]).drop_duplicates()
    return df.iloc[:size[0]]

In [51]:
code = np[6, 0, 9]
trial = np[1,2,3]
result = score(trial, code)
result

array([0, 0])

In [5]:
def fitness(trial, guesses):
    print(guesses)
    X, Y = np.array([np.abs(score(trial, np.array(guess))-guess_result) for (guess, guess_result) in guesses]).sum(axis=0)
    return X + Y - len(trial)

def stringify(lst):
    return "".join(map(str, lst))

def destringify(lst):
    return list(map(int, list(lst)))

code = np[6, 0, 9]
trial = np[1,2,3]
result = score(trial, code)
turn = 0

print(trial, result, mark(trial, code))

guesses = []
# guesses = [
#     ([1,2,6],[0,1]),
#     ([6,3,0],[1,1]),
#     ([6,0,4],[2,0]),
#     ([6,0,5],[2,0])
    
# ]

while (result != [3,0]).any():
    population = generatePopulation().values

    Ei = []
    h, k = 1, 0

    while len(Ei) <= len(population) and h <= 100:
        #print(population)
        h += 1
        children = np.array(reduce(lambda lst,i:lst+[permute((lambda gene:mutate(gene) if np.random.random()<0.03 else gene)(crossover(population[i],population[i+1])))], range(len(population)-1), [])+[population[-1]])
        pop_score = sorted([(fitness(c, guesses), c) for c in children], key=lambda x:x[0])
        eligibles = [(score, e) for (score, e) in pop_score if score == 0]
        print(eligibles)
        if not len(eligibles): continue
        stringligibles = set([stringify(i[-1]) for i in eligibles])
        Ei_new = set(map(stringify, Ei))
        Ei_new = list(Ei_new - stringligibles)
        Ei_new.extend(list(map(stringify, np.random.choice(10,size=(len(Ei)-len(Ei_new), 3)).tolist())))
        Ei_new = list(set(Ei_new) | stringligibles)[:len(population)]
        Ei = list(map(destringify, Ei_new))
        population = np.vstack((np.array([i[-1] for i in eligibles]),np.random.choice(10,size=(len(Ei)-len(Ei_new), 3))))

    print("Ei", end="")
    if len(Ei) == 0:
        print(" is 0")
        continue
    print(" ", Ei)
    test = Ei.pop()
    while code in [c for (c, r) in guesses]:
        test = Ei.pop()
    
    turn += 1
    
    result = score(test, code)
    print(code, result, mark(test, code))
    guesses.append((code, result))
    
    

[1 2 3] [0 0] ['.' '.' '.']
[]


TypeError: cannot unpack non-iterable numpy.int32 object

In [3]:
import np
import re
import pandas as pd
from functools import reduce


def p(*size): return np.random.random(size=size if len(size) else 1)

def rand(*size): return np.random.randint(10, size=size if len(size) else 1)

def shuffle(arr):
    cp = arr.copy()
    np.random.shuffle(cp)
    return cp

class Chromosome:
    def __init__(self, *args):
        if len(args) == 1: args = args[0]
        self.values = np.array(list(map(int, args))).flatten()
        self.slots = len(self.values)
        
    def copy(self):
        return Chromosome(*self.values.tolist())
    
    def __str__(self):
        return f"Chromosome("+", ".join(map(str, self.values))+")"
    
    def __len__(self):
        return self.slots
    
    def __getitem__(self, i):
        return self.values[i]
    
    def __setitem__(self, i, obj):
        self.values[i] = obj
        
    def randSlots(self, size=1):
        return np.random.randint(self.slots-1, size=size)
        
    def tolist(self):
        return self.values.tolist()

    def stringify(self):
        return "".join(map(str, self.tolist()))
    
    @classmethod
    def destringify(cls, string):
        return cls(*map(int, string))
    
    def crossover(self, other):
        return Chromosome(np.where(p(self.slots)<0.5, self.values, other.values))
    
    def mutate(self):
        code = self.copy()
        if p() < 0.03:
            code[code.randSlots()] = rand()
        return code
    
    def invert(self):
        code = self.copy()
        if p() < 0.03:
            a, b = sorted(tuple(code.randSlots(2)))
            code[a:b] = code[a:b][::-1]
        return code
    
    def scramble(self):
        code = self.copy()
        if p() < 0.03:
            a, b = sorted(tuple(code.randSlots(2)))
            code[a:b] = shuffle(code[a:b])
        return code
    
    def swap(self,a,b):
        self[a], self[b] = self[b], self[a]

    def permute(self):
        code = self.copy()
        for i in range(code.slots):
            if p() < 0.03: # very low
                code.swap(*code.randSlots(2))
        return code
    
    def __hash__(self):
        return int(self.stringify())
    
    def __eq__(self, other):
        return (self.values == other.values).sum()
    
    def __ne__(self, other):
        return self.values != other.values
    
    def within(self, other):
        return np.isin(self.values, other)
    
    def whites(self, guess):
        return guess.within(list(set(self[guess != self]) & set(guess[guess != self]))).astype(int)
       
    def blacks(self, guess):
        return (self.values == guess.values)
    
    def score(self, guess):
        return np[self == guess, guess.whites(self).sum()]
    
    def mark(self, guess):
        output = np.full_like(self.values, ".", dtype=str)
        output[guess.whites(self)] = "W"
        output[guess.blacks(self)] = "B"
        return output
    
    def fitness(self, guesses):
        return np.array([self.score(guess)-guess_result for (guess, guess_result) in guesses]).sum() - self.slots

    
class ChromoSet(list):
    """
    Can be used for guesses, genotype, eligibles etc
    """
    def __init__(self, values=[]):
        self.values = [Chromosome(candidate) for candidate in np.array(values).tolist()]
        super().__init__(self.values)
        
    def __gt__(self, other):
        return len(self) > len(other)
    
    def __ge__(self, other):
        return len(self) >= len(other)
    
    def __lt__(self, other):
        return len(self) < len(other)
    
    def __le__(self, other):
        return len(self) <= len(other)
    
    def filter(self, f):
        return Elite([chromosome for chromosome in self if f(chromosome)])
        

class Elite(ChromoSet):
    def __add__(self, other):
        return Elite(list(set(self) | set(other)))
    def __sub__(self, other):
        return Elite(list(set(self) - set(other)))
        
        
class Genotype(ChromoSet):
    def __init__(self, size, slots, initial=[]):
        self.size = size
        self.slots = slots
        df = pd.DataFrame(initial)
        while len(df) < size:
            pop = rand(size-len(df), slots)
            #print(df)
            #print(pop)
            #print(np.vstack((df, pop)))
            print(df)
            df = pd.concat([df, pd.DataFrame(pop)]).drop_duplicates()
        
        super().__init__(df.iloc[:size].values)
        
    def reproduce(self):
        lst = []
        for i, chromosome in enumerate(self):
            if i == len(self)-1:
                lst.append(chromosome)
                break
            lst.append(chromosome.crossover(self[i+1]).mutate().invert().scramble().permute())
        
        return Genotype(len(lst), len(lst[0]), initial=list(map(Chromosome.tolist, lst)))
        
    def train(self, guesses):
        Ei = Elite()
        cnt = 0
        while h <= 100 and Ei <= self:
            children = self.reproduce().filter(lambda child: not child.fitness(guesses))
            Ei = Ei + children
            
            
            
def train(guesses):
    pop = Genotype(150, 3)
    Ei = Elite()
    cnt = 0
    while cnt <= 100 and Ei <= pop:
        children = pop.reproduce().filter(lambda child: not child.fitness(guesses))
        if len(children):
            Ei = Ei + children
            pop = Genotype(150, 3, initial=Ei)
    
    return Ei


guesses = [
    (Chromosome([1,2,6]),np[0,1]),
    (Chromosome([6,3,0]),np[1,1]),
    (Chromosome([6,0,4]),np[2,0]),
    (Chromosome([6,0,5]),np[2,0])

]

train(guesses)

Empty DataFrame
Columns: []
Index: []
     0  1  2
0    6  5  3
1    4  4  9
2    1  9  7
3    6  0  0
4    7  2  5
..  .. .. ..
145  8  4  8
146  3  8  6
147  1  5  1
148  9  8  9
149  3  1  1

[142 rows x 3 columns]
    0  1  2
0   6  5  3
1   4  4  9
2   1  9  7
3   6  0  0
4   7  2  5
.. .. .. ..
2   4  6  6
3   2  1  3
5   7  9  6
6   1  6  1
7   4  5  7

[148 rows x 3 columns]
           0
0  [0, 6, 6]


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()